# Whisper48

<font size="3">基于 faster-whisper 的 Google Colab 云端部署。   
\
请按提示依次执行以下单元格，建议在开始前先将需要转录的音频文件上传到谷歌网盘中。  
Please run the following cells in order according to the help text. It is suggested to upload your audio file to Google Drive first before you start.  
\
联系作者/Contact me: ifeimi48@gmail.com.<font size="3">  

In [ ]:
#@markdown **1.1 挂载谷歌云盘/Mount Google Drive (approx. 0.5 min)** 
#@markdown **</br><font size="2">【重要】:** 请确认在"修改"->"笔记本设置"->"硬件加速器"中选择了GPU作为硬件加速。
#@markdown **</br>【IMPORTANT】:** Make sure you have selected GPU as hardware accelerator in "Runtime" -> "Change runtime type".</font><br/>
from google.colab import drive
from google.colab import files
import os
import logging
from IPython.display import clear_output 

clear_output()
print('Please allow connecting to Google Drive in the pop-up window')
print('请在弹出窗口中选择同意挂载谷歌云盘')
drive.mount('/drive')
print('Google Drive mounted, please proceed to next step')
print('谷歌云盘挂载完毕，请执行下一步')

In [ ]:
#@markdown **1.2 配置运行环境/Setup environment (approx. 1 min)**

! pip3 install ipytree -q
! pip3 install faster-whisper -q
clear_output()
#! pip3 show torch
#! pip3 show torchvision
print('Environment is ready, please proceed to next step')
print('运行环境配置完毕，请执行下一步')

In [ ]:
#@markdown **1.3 从谷歌云盘选择文件/Select File From Google Drive (0 min)**
# @markdown <br/><font size="2">从网盘目录中选择要转换的文件(视频/音频），单击选中文件，点击'Select'按钮以确认。
# @markdown <br/>Navigate to the file you want to transcribe, left-click to highlight the file, then click 'Select' button to confirm.
# @markdown <br/>若到这一步才上传文件到谷歌网盘，则重复执行本单元格以刷新文件列表。
# @markdown <br/>If the file was not uploaded until this cell, execute this cell again to refresh.</font>



from google.colab import output 
from utils import file_tree

output.enable_custom_widget_manager()
use_drive = True
global drive_dir
drive_dir = ''


tree= file_tree()
tree


In [ ]:
# @markdown **2.1 参数设置/Parameter setting (0 min)**
# @markdown </br></br><font size="3">**2.1.1 选择上传的文件类型(视频-video/音频-audio）/ Select the type of the file uploaded.**</font><br/>
file_type = "audio"  # @param ["audio","video"]

# @markdown <font size="3">**2.1.2 选择模型和语言 / Model size and language.**</font><br/>
model_size = "large-v3"  # @param ["base","small","medium", "large-v1","large-v2","large-v3"]
language = "ja"  # @param ["ja","zh","en","fr", "de","es","it","pt","ru"]

# @markdown <font size="3">**2.1.3 高级设置 / Andvanced settings** (如果你不确定这是做什么的，请保持默认)</font>
chunk_size = 5  # @param {type:"integer"}
length_penalty = 1.0  # @param {type:"number"}

In [ ]:
#@markdown **2.2 运行Whisper/Run Whisper (approx. 5-20 min)**
#@markdown </br><font size="2">完成后srt文件将自动下载到谷歌云盘中
#@markdown </br>SRT file will be downloaded automatically after finishing.</font><br/>

import os
import torch
from faster_whisper import WhisperModel
import time
from urllib.parse import quote_plus
import gc
# assert file_name != ""
# assert language != ""

output_dir = os.path.dirname(drive_dir)
try:
    file_name = drive_dir
    file_basename = file_name.split('.')[0]
    output_dir = os.path.dirname(drive_dir)
except Exception as e:
    print(f'error: {e}')


if file_type == "video":
  print('提取音频中 Extracting audio from video file...')
  os.system(f'ffmpeg -i {file_name} -ar 16000 -ac 1 -c:a pcm_s16le {file_basename}.wav')
  print('提取完毕 Done.') 
  audio_file = f'{file_basename}.wav'
else:
    audio_file = f'{file_name}'

device = "cuda"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

print('加载模型 Loading model...')
model = WhisperModel(model_size, device=device, compute_type=compute_type)

# Original whisper transcribe
tic = time.time()
print('识别中 Transcribe in progress...')
segments, info = model.transcribe(audio_file, language=language, log_progress=True, length_penalty=length_penalty, beam_size=5, vad_filter=True, chunk_length=chunk_size)
segments = list(segments)

print('语音转录完成 Transcribing completed')
gc.collect(); torch.cuda.empty_cache(); del model

#Write SRT file
filename_srt = file_basename + ".srt"

def _format_srt_time(t: float) -> str:
    hours = int(t // 3600)
    minutes = int((t % 3600) // 60)
    seconds = int(t % 60)
    milliseconds = int((t - int(t)) * 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

with open(filename_srt, "w", encoding="utf-8") as f:
    for i, segment in enumerate(segments, start=1):
        start_ts = _format_srt_time(segment.start)
        end_ts = _format_srt_time(segment.end)
        text = segment.text.strip().replace("\n", " ")
        f.write(f"{i}\n{start_ts} --> {end_ts}\n{text}\n\n")
files.download(filename_srt)

print('字幕生成完毕 Subtitle generated!')